# 301 Calculate CO2 drawdown

In this notebook, we assess the CO2 drawdown per scenario.

In [1]:
import pyam
import pandas as pd
import numpy as np
import scmdata
from utils import (
    assign_warming_levels,
    _prep_df_for_subtract
)

from pathlib import Path
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/scmdata/database/_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


## Read in the necessary data

For this analysis, we want to compare the temperature drawdown across two cases:
1. The original multi-gas pathway run through MAGICC for all gases
2. The multi-gas pathway flat lined after net zero run through MAGICC for all gases

In [2]:
df_original_ghg = pyam.IamDataFrame(
    Path(
        '../data/201_original_all_ghg_temps.csv'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/201_original_all_ghg_temps.csv


In [3]:
df_netzero_ghg = pyam.IamDataFrame(
    Path(
        '../data/202_netzeroCO2_all_ghg_temps.csv'
    )
)

pyam.core - INFO: Reading file ../data/202_netzeroCO2_all_ghg_temps.csv


In [4]:
df_netzero_ghg.scenario

['DeepElec_SSP2_ HighRE_Budg900_NZCO2',
 'EN_NPi2020_400f_lowBECCS_NZCO2',
 'SusDev_SDP-PkBudg1000_NZCO2']

## Data processing

Step 1: Calculate the drawdown for each dataframe

In [5]:
drawdown_original = assign_warming_levels(
    df_original_ghg
)

In [6]:
drawdown_netzero = assign_warming_levels(
    df_netzero_ghg
)

Step 2: Rename the netzero scenario column so that we can subtract the two dataframes.

In [7]:
drawdown_netzero_for_metrics = drawdown_netzero.copy()

In [8]:
drawdown_original_for_metrics = drawdown_original.copy()

In [9]:
drawdown_netzero_for_metrics.loc[:,'scenario'] = (
    drawdown_netzero_for_metrics.loc[:,'scenario']
    .apply(
        lambda x: x.replace('_NZCO2','')
    )
)

Step 3: Pull out the necessary columns for calculations

In [10]:
drawdown_original_for_metrics = _prep_df_for_subtract(
    drawdown_original_for_metrics,
    cols='drawdown'
)

In [11]:
drawdown_netzero_for_metrics = _prep_df_for_subtract(
    drawdown_netzero_for_metrics,
    cols='drawdown'
)

Step 4: Calculate the drawdown due to CO2

In [12]:
drawdown_co2 = pd.DataFrame(
    drawdown_netzero_for_metrics
    -
    drawdown_original_for_metrics
)

Step 5: Rename the drawdown column

In [13]:
drawdown_co2.rename(
    columns={
        'drawdown':'drawdown_co2'
    },
    inplace=True
)

## Save out the data

In [14]:
drawdown_co2.to_csv(
    Path(
        '../data/301_co2_drawdown.csv'
    )
)